In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np



# empty data frame with columns

In [ ]:
apple_forum = pd.DataFrame({'title' : [], 'date': [], 'user':[], 'nb_views':[], 'nb_replies':[], 'link':[]})

In [ ]:
html = requests.get("https://discussions.apple.com/community/iphone/iphone_hardware").text
soup = BeautifulSoup(html, 'html.parser')
bs_object = soup.find_all('div', attrs={"class":u"discussion-description"})

## find all the elements in the page corresponding to a thread

In [ ]:
for i in range(1,3):
    html = requests.get("https://discussions.apple.com/community/iphone/iphone_hardware?page="+str(i)).text
    soup = BeautifulSoup(html, 'html.parser')
    bs_object = soup.find_all('div', attrs={"class":u"discussion-description"})

    for s,index in zip(bs_object,range(len(bs_object))): 
    #links to threads
        apple_forum.loc[index+15*(i-1),'link'] = s.find('div', attrs={"class":u"discussion-title"}).find('a').get('href')
    
    #question text
        apple_forum.loc[index+15*(i-1),'title'] = s.find('div', attrs={"class":u"discussion-title"}).find('a') \
        .find('span', attrs={"class":u"discussion-description-inner-responsive u-hide--desktop"}).get_text().strip()
    
    #view number
        apple_forum.loc[index+15*(i-1),'nb_views'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find('div', attrs={"class":u"cell discussion-views u-hide--small"}).get_text().strip()
    
    #user name
        apple_forum.loc[index+15*(i-1),'user'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find('div', attrs={"class":u"cell discussion-name u-hide--small"}).get_text().strip()
    
    #latest reply and user replying
        apple_forum.loc[index+15*(i-1),'date'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find('div', attrs={"class":u"cell align-metadata"}).get_text().strip()
    
    #latest reply and user replying
        apple_forum.loc[index+15*(i-1),'nb_replies'] = s.find('div', attrs={"class":u"discussion-metadata"}) \
        .find_all('div', attrs={"class":"cell align-metadata"})[1].get_text().strip()

In [ ]:
apple_forum

In [293]:
messages = pd.DataFrame({'thread_id' : [], 'date': [], 'message_id':[], 'message_text':[], \
                         'poster':[], 'interest':[], 'question':[], 'model':[]})

In [295]:

i=0
test = ["/thread/8163518","/thread/8241667"]
for link in test:
    html = requests.get("https://discussions.apple.com"+link).text
    soup = BeautifulSoup(html, 'html.parser')
    bs_object = soup.find('div', attrs={"class":u"j-column-wrap-l"})
    
    length = len(bs_object.find_all('div', attrs={"class":u"jive-rendered-content"}))
    
    l=0
    while l < length:
        if l == 0:
            messages.loc[i,'question'] = True
            
            ### poster name 
            messages.loc[i,'poster'] = bs_object.find('div', attrs={"class":u"j-post-author"}).find('a').text.strip()
    
            ### poster model name / OS
            try:
                messages.loc[i,'model'] = bs_object.find('p', attrs={"class":u"jive-model-info-display"}).text.strip()
            except: 
                messages.loc[i,'model'] = "not disclosed"
            ### poste date
            messages.loc[i,'date'] = bs_object.find('p', attrs={"class":u"meta-posted"}).text.strip()

            ###me too
            messages.loc[i,'interest'] = bs_object.find('a', attrs={"class":u" jive-acclaim-likelink black-button "}).text.strip()
            
            ###question id
            messages.loc[i,'message_id'] = bs_object.find('', attrs={"class":u" jive-acclaim-likelink black-button "}).get("data-object-id")
            
        else:
            messages.loc[i,'question'] = False
            
            permalink = bs_object.find_all('a', attrs={"class":u"discussionPermaLink"})
            messages.loc[i,'message_id'] = permalink[l-1].get("href")
            
            #user 
            users = bs_object.find_all('span', attrs={"class":u"username"})
            messages.loc[i,'poster'] = users[l-1].text.strip()

            #replying to + date
            dates = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
            messages.loc[i,'date'] = dates[l-1].text.strip()
            
            #helpful answer count
            helpful = bs_object.find_all('span', attrs={"class":u"js-acclaim-container acclaim-container acclaim-like-container "})
            messages.loc[i,'interest'] = helpful[l-1].get("data-likes")
            
        messages.loc[i,'thread_id'] = link
        m_text = bs_object.find_all('div', attrs={"class":u"jive-rendered-content"})
        messages.loc[i,"message_text"] = m_text[l].text.strip()
        i+=1
        l+=1
        

In [296]:
messages

,date,interest,message_id,message_text,model,poster,question,thread_id
0,"Posted on Nov 16, 2017 5:57 AM",I have this question too (6),32574951,All my apps are reallyt laggy.... Was expecti...,"iPhone X, iOS 11.1",SkooterUSA,True,/thread/8163518
1,"Nov 16, 2017 6:05 AM\n\n\nin response to Skoot...",1,/message/32574968#message32574968,You can give feedback here: Feedback - iPhone ...,NaN,anonyme4321,False,/thread/8163518
2,"Jan 11, 2018 5:35 AM\n\n\nin response to Skoot...",0,/message/32852786#message32852786,I second that. Updated to iOS 11.2 this mornin...,NaN,One of Three,False,/thread/8163518
3,"Jan 16, 2018 2:56 AM\n\n\nin response to One o...",0,/message/32877235#message32877235,"If the entire phone is slow, and you upgraded ...",NaN,anonyme4321,False,/thread/8163518
4,"Jan 16, 2018 3:10 AM\n\n\nin response to anony...",0,/message/32877304#message32877304,"OK, so I was browsing possible solutions onlin...",NaN,One of Three,False,/thread/8163518
5,"Posted on Jan 16, 2018 4:46 AM",I have this question too,32877503,I live in Bournemouth England and wish to get ...,not disclosed,Colin62,True,/thread/8241667
6,"Jan 16, 2018 4:56 AM\n\n\nin response to Colin62",0,/message/32877594#message32877594,You should contact Apple directly. A Message t...,NaN,Roger Wilmut1,False,/thread/8241667
7,"Jan 16, 2018 5:49 AM\n\n\nin response to Colin62",0,/message/32877682#message32877682,,NaN,Briansyddall,False,/thread/8241667
8,"Jan 16, 2018 5:57 AM\n\n\nin response to Colin62",0,/message/32877683#message32877683,Hi Apple repair centre Rodmell Lane Bournemout...,NaN,Briansyddall,False,/thread/8241667
9,"Jan 16, 2018 6:50 AM\n\n\nin response to Brian...",0,/message/32878127#message32878127,Do we know whether this is an Apple Authorised...,NaN,Roger Wilmut1,False,/thread/8241667


In [264]:
bs_object.find('span', attrs={"class":u"jive-model-info-display"})

In [ ]:
html = requests.get("https://discussions.apple.com/thread/8163518").text
soup = BeautifulSoup(html, 'html.parser')
bs_object = soup.find('div', attrs={"class":u"j-column-wrap-l"})

In [ ]:
### poster name 
bs_object.find('div', attrs={"class":u"j-post-author"}).find('a').text.strip()
    
### poster model name / OS
bs_object.find('p', attrs={"class":u"jive-model-info-display"}).text.strip()

### poste date
bs_object.find('p', attrs={"class":u"meta-posted"}).text.strip()

###me too
bs_object.find('a', attrs={"class":u" jive-acclaim-likelink black-button "}).text.strip()

###question id
bs_object.find('', attrs={"class":u" jive-acclaim-likelink black-button "}).get("data-object-id")

### replying users
table = bs_object.find_all('span', attrs={"class":u"username"})
for x in table:
    x.text.strip()
    
#initial message + reply text: 
table = bs_object.find_all('div', attrs={"class":u"jive-rendered-content"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

#replying to
table = bs_object.find_all('span', attrs={"class":u"font-color-meta-light j-thread-replyto"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

##perma link to replies
table = bs_object.find_all('a', attrs={"class":u"discussionPermaLink"})
for x in range(0,len(table)):
    print(x)
    print(table[x].get("href"))

#replying to + date
table = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())
    
# helpful
table = bs_object.find_all('span', attrs={"class":u"js-acclaim-container acclaim-container acclaim-like-container "})
for x in table:
    print(x.get("data-likes"))

In [272]:
#replying to + date
table = bs_object.find_all('span', attrs={"class":u"j-inresponse-to"})
for x in range(0,len(table)):
    print(x)
    print(table[x].text.strip())

0
Jan 16, 2018 4:56 AM


in response to Colin62
1
Jan 16, 2018 5:49 AM


in response to Colin62
2
Jan 16, 2018 5:57 AM


in response to Colin62
3
Jan 16, 2018 6:50 AM


in response to Briansyddall
4
Jan 16, 2018 7:14 AM


in response to Colin62
5
Jan 16, 2018 8:03 AM


in response to Michael Black
6
Jan 16, 2018 8:14 AM


in response to Roger Wilmut1


In [292]:
table = bs_object.find_all('span', attrs={"class":u"username"})
for x in table:
    print(x.text.strip())

Roger Wilmut1
Briansyddall
Briansyddall
Roger Wilmut1
Michael Black
Roger Wilmut1
Michael Black


In [279]:
print(bs_object.prettify())

<div class="j-column-wrap-l">
 <div class="j-column j-column-l lg-margin">
  <!-- BEGIN thread messages -->
  <div class="jive-thread-messages" id="jive-thread-messages-container" itemscope="" itemtype="http://schema.org/Question" role="main">
   <!-- BEGIN thread info -->
   <div class="jive-thread-info clearfix j-rc4">
   </div>
   <!-- END thread info -->
   <!-- BEGIN original thread post / removed && (!paginator.previousPage) -->
   <div class="jive-content j-op j-rc4 " role="article">
    <div class="j-thread-post j-rc4 ">
     <!-- commenting the span as part of new design -->
     <!-- <span class="j-thread-pointer j-ui-elem"></span> -->
     <a name="8241667">
     </a>
     <a name="32877503">
     </a>
     <header class="js-original-header apple-overflow-hidden">
      <a alt="" aria-hidden="true" class="j-avatar jiveTT-hover-user u-hide--small" data-avatarid="8425853" data-externalid="" data-userid="8425853" data-username="Colin62" href="/people/Colin62" role="button" tabi